In [1]:
# This script provides some reporting on the resolver data from identifiers.org
#
# Author: Manuel Bernal Llinares <mbdebian@gmail.com>

In [2]:
import time
import random
import pandas
import requests
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Endpoint from where the information is coming
identifiersorg_resolver_data_url = "https://identifiers.org/rest/collections/expand"
metadata_service_endpoint_from_url = "http://localhost:8082/getMetadataForUrl"

In [ ]:
# Initialize pseudo-random number generator
random.seed(time.time())

In [ ]:
# Request Helper
def make_rest_request_content_type_json(url):
    # TODO - Magic number here!!!
    n_attempts = 42
    response = None
    while n_attempts:
        n_attempts -= 1
        try:
            response = requests.get(url, headers={"Content-Type": "application/json"})
        except Exception as e:
            # Any possible exception counts towards the attempt counter
            # Random wait - TODO - Another magic number!!!
            time.sleep(random.randint(30))
            continue
        if response.ok:
            return response.json()
        # Random wait - TODO - Another magic number!!!
        time.sleep(random.randint(10))
    response.raise_for_status()

In [ ]:
# Get the resolver data
resolver_dump = make_rest_request_content_type_json(identifiersorg_resolver_data_url)

In [ ]:
# Workout how many prefixes there are in identifiers.org
prefixes = [pid_entry['prefix'] for pid_entry in resolver_dump]

In [ ]:
prefixes

In [ ]:
print("There are #{} Compact ID prefixes registered in identifiers.org".format(len(prefixes)))

In [ ]:
# Check the distribution of resources
resource_prefixes = []
for pid_entry in resolver_dump:
    if 'resources' in pid_entry:
        for resource in pid_entry['resources']:
            if 'resourcePrefix' in resource:
                resource_prefixes.append(resource['resourcePrefix'])
            else:
                print("NO RESOURCE PREFIX FOR: PID Entry Name '{}', Resource Information '{}'".format(pid_entry['name'], resource['info']))


In [ ]:
resource_prefixes

In [ ]:
resource_prefixes_distribution = Counter(resource_prefixes)

In [ ]:
resource_prefixes_distribution

In [ ]:
print("There #{} Resource Selectors in identifiers.org".format(len(resource_prefixes_distribution.keys())))

In [ ]:
labels = sorted(resource_prefixes_distribution.keys())
values = [resource_prefixes_distribution.get(key) for key in labels]

In [ ]:
# I don't like this Pie Chart
fig1, ax1 = plt.subplots()
fig1.set_size_inches(18, 5)
ax1.bar(labels, values, 1/1.5, color="blue")
#ax1.pie(values, labels=labels, shadow=True, startangle=90)
#ax1.axis('equal')
plt.show()

In [ ]:
# Create a report on metadata